In [1]:
import numpy as np
import ipdb

### Prepare a Toy Dataset

In [2]:
X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0],[1],[1],[0]])

### Simple Neural Network 

In [487]:
np.random.seed(1)

# randomly initialize our weights with mean 0
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1

for j in range(50000):

    # Feed forward through layers 0, 1, and 2
    l0 = X
    l1 = sigmoid(np.dot(l0,syn0))
    l2 = sigmoid(np.dot(l1,syn1))

    # how much did we miss the target value?
    l2_error = y - l2  
 
    if (j% 10000) == 0:
        print("Error:" + str(np.mean(np.abs(l2_error))))
        
    # in what direction is the target value?
    # were we really sure? if so, don't change too much.
    l2_delta = l2_error*(l2* (1-l2))      
    
    # how much did each l1 value contribute to the l2 error (according to the weights)?
    l1_error = l2_delta.dot(syn1.T)
    
    
    # in what direction is the target l1?
    # were we really sure? if so, don't change too much.
    l1_delta = l1_error * (l1*(1-l1))
   
    syn1 += l1.T.dot(l2_delta)
    syn0 += l0.T.dot(l1_delta)


Error:0.496410031903
Error:0.00858452565325
Error:0.00578945986251
Error:0.00462917677677
Error:0.00395876528027


### Now our plan to make this piece more abstract and make it a way for easy extension. The objective here to create a sequential class where the network is create with lot of abstraction. For Example                                                model = Sequential([Dense(4),Dense(1,activation = sigmoid) ], epochs =10000, lr= 0.01)(X,y)                                          model.fit()                                                                                                                                                                                    Sequential takes 2 parameters layers list, no of iteration(epochs) , learning rate                                                              Dense layer is the linear layer followed activation, the activation default is Sigmoid can be passed if anything     different. The core idea here to explore the popular activation, loss and extend the neural network to use in a efficient way     

 
  
   
  
 






In [488]:
#This cell block has the List of Activation Functions
def sigmoid(h):
    return 1/(1+np.exp(-h)) 

def relu(h):
    return h * (h >0)

def tanh(h):
    pass

def leakyrelu(h):
    pass

def elu(h):
    pass

In [489]:
#This Cell Block has List of Loss functions
def binaryLoss(y, p, c= None):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))    

In [490]:
class Sequential:
    def __init__(self, layers, epochs, lr, loss = binaryLoss):
        self.layers,self.epochs, self.lr,self.loss = layers,epochs, lr ,loss
        
    def __call__(self, X, y): 
        #assign weights
        self.X,self.y = X,y    
        
        inputdim = X.shape[1]
        np.random.seed(0)     
        #initialize layers
        for layer in self.layers:
            inputdim = layer(inputdim,self.lr) 
            
        return self    
          
    def fit(self):  
        for i in range(self.epochs):
            h = self.X
          
            #compute hidden units
            for layer in self.layers:                 
                h = layer.forward(h)              
        
            loss = self.loss(self.y, h)  
            
        
            error = self.y - h             
            if((i%1000) == 0):
                errorrate = np.mean(np.abs(error))                  
                print(f'Epoch# {i} Loss:{loss} Error: {errorrate}') 
            
            #back propagate the error - this formula is influenced by andrew ng course  
            for layer in reversed(self.layers):                                 
                error = layer.backward(error)          
               
                
            for i in reversed(range(0,len(self.layers))): 
                h = self.X if i == 0 else self.layers[i-1].h
                self.layers[i].step(h)

In [496]:
from abc import ABC, abstractmethod

class Layer(ABC):
    def __init__(self, outdim, activation = sigmoid): 
        self.outdim = outdim
        self.activation = activation
          
        
    def __call__(self, inputdim, lr):  
        self.inputdim,self.lr  = inputdim,lr
        self.w = (2*np.random.random((self.inputdim, self.outdim))) - 1
        self.lr = lr
        return self.outdim
        
   
    def forward(self, x):
        self.h = self.activation(x)    
        return self.h
    
    def backward(self, error):   
        #given an output value from a neuron, we need to calculate it’s slope.
        #We are using the sigmoid transfer function, the derivative of which can be calculated as follows:
        #h * (1- h)
        
        #If you look at below equation - error = ( output - input)   (self.h * (1 - self.h)) - slope of output
        #Apply the derivative of our sigmoid activation function to the output layer error
        self.delta = error * (self.h * (1 - self.h))
        
        #Use the delta output  to figure out how much our hidden layer contributed to the output error 
        #by performing a dot product with our weight matrix
        error = self.delta.dot(self.w.T)         
        return error
    
    def step(self,h):      
        self.w += h.T.dot(self.delta)        
         

In [497]:
class Dense(Layer):
    def __init__(self, outdim, activation = sigmoid):        
        super().__init__(outdim,activation)        
      
        
    def forward(self,x):        
        #linear 
        h = np.dot(x, self.w) 
        return super().forward(h)

In [498]:
model = Sequential([    
    Dense(10),    
    Dense(1)    
], epochs =3001, lr= 0.001)(X,y)

In [499]:
model.fit()

#we can see the loss geeting reduced on each iterations. 

Epoch# 0 Loss:0.8095493812635659 Error: 0.49609260484007356
Epoch# 1000 Loss:0.04179670525777059 Error: 0.0409134589822716
Epoch# 2000 Loss:0.023069436970873734 Error: 0.02279930553572503
Epoch# 3000 Loss:0.0172455911627992 Error: 0.017094520819156633


### Let try to extent the sequential class to better predict the hand written digits in a next session